<a href="https://colab.research.google.com/github/giorgiosld/Natural-Language-Processing/blob/main/labs/lab5/T_725_Lab05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T-725 Natural Language Processing: Lab 5
In today's lab, we will be working with neural networks, using GRUs and Transformers for text generation.

To begin with, do the following:
* Select `"File" > "Save a copy in Drive"` to create a local copy of this notebook that you can edit.
* **Select `"Runtime" > "Change runtime type"`, and make sure that you have "Hardware accelerator" set to "GPU"**
* Select `"Runtime" > "Run all"` to run the code in this notebook.

In [1]:
import os
import warnings

# Suppress some warnings from TensorFlow about deprecated functions
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

## Generating text with neural networks
Let's create a neural language model and use it to generate some text. This time, we will use character embeddings rather than word embeddings. They are created in exactly the same way, and are often used together in neural network-based models. One benefit of using character embeddings is that we can generate words that our model has never seen before.

The model takes as input a sequence of characters and predicts which character is most likely to follow. We will generate text by repeatedly predicting and appending the next character to a string. First, however, we need some text to train it on.


In [2]:
# Based on the following tutorial:
# https://www.tensorflow.org/tutorials/text/text_generation

import tensorflow as tf
import numpy as np
import os
import time

# Let's download some text by Shakespeare to train our model
url = 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'
path_to_file = tf.keras.utils.get_file('shakespeare.txt', url)

with open(path_to_file, encoding='utf-8') as f:
  shakespeare = f.read()

print("First 250 characters:")
print(shakespeare[:250])

print ("Length of text: {:,} characters".format(len(shakespeare)))

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step
First 250 characters:
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

Length of text: 1,115,394 characters


Now we can create training examples for our model. Each example will be a pair of strings: one input string containing 100 characters, and a target string that is one character ahead. For example, the first pair we create is:

**Input string**:  `'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'`

**Target string**: `'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '`

However, before we can start training, we need to convert our text into a list of integers, where each integer represents a different character. For example, "First Citizen" becomes:

```
Character:   F   i   r   s   t      C   i   t   i   z   e   n
Integer:   [18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52]
```

In [3]:
# Hyper-parameters:

BATCH_SIZE = 64  # Batch size
BUFFER_SIZE = 10000  # Buffer size to shuffle the dataset
SEQUENCE_LENGTH = 100  # Length of input sequence
EMBEDDING_DIMENSION = 65  # Embedding dimension
RNN_UNITS = 1024  # Number of RNN units

In [4]:
def split_input_target(chunk):
  # Create (input_string, output_string) pairs
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

def prepare_text(text):
  # The unique characters in the file
  vocab = sorted(set(text))
  print ('{} unique characters'.format(len(vocab)))

  # Creating a mapping from unique characters to indices
  char_map = {
      'char_to_index': {char: index for index, char in enumerate(vocab)},
      'index_to_char': np.array(vocab)
  }

  text_as_int = np.array([char_map['char_to_index'][c] for c in text])

  # The maximum length sentence we want for a single input in characters
  seq_length = SEQUENCE_LENGTH
  examples_per_epoch = len(text) // (seq_length+1)

  # Create training examples / targets
  char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
  sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)
  dataset = sequences.map(split_input_target)

  # (TF data is designed to work with possibly infinite sequences,
  # so it doesn't attempt to shuffle the entire sequence in memory. Instead,
  # it maintains a buffer in which it shuffles elements).
  dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

  return dataset, vocab, examples_per_epoch, char_map

Now we can create and train the neural network.

In [5]:
import os

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)


def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size,
                                embedding_dim),
      tf.keras.layers.GRU(rnn_units,
                          return_sequences=True,
                          recurrent_initializer='glorot_uniform',
                          stateful=True),
      tf.keras.layers.Dense(vocab_size)
  ])

  return model

In [6]:
def create_model(text, epochs=3):
  dataset, vocab, examples_per_epoch, char_map = prepare_text(text)

  train_model = build_model(len(vocab), EMBEDDING_DIMENSION, RNN_UNITS, BATCH_SIZE)
  train_model.compile(optimizer='adam', loss=loss)

  train_model.fit(dataset, epochs=epochs)

  pred_model = build_model(len(vocab), EMBEDDING_DIMENSION, RNN_UNITS, batch_size=1)
  pred_model.build(input_shape=(1, 100))
  pred_model.set_weights(train_model.get_weights())

  return pred_model, char_map

In [7]:
shakes_model, shakes_chars = create_model(shakespeare, epochs=3)

65 unique characters
Epoch 1/3
172/172 ━━━━━━━━━━━━━━━━━━━━ 22s 53ms/step - loss: 3.2147
Epoch 2/3
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 48ms/step - loss: 2.0777
Epoch 3/3
172/172 ━━━━━━━━━━━━━━━━━━━━ 12s 47ms/step - loss: 1.7864


Now that we've trained our model, we can finally use it to generate some text. The following function takes a model and a string as input, and continually predicts and appends the next character to the string until it becomes 1,000 characters long.

In [8]:
def generate_text(model, char_map, start_string, temperature=1.0):
  # Evaluation step (generating text using the learned model)
  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  if not start_string:
    print("start_string can't be empty")
    return ""

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char_map['char_to_index'][s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Here batch size == 1
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(char_map['index_to_char'][predicted_id])

  return (start_string + ''.join(text_generated))

Let's generate some text!

In [9]:
print(generate_text(shakes_model, shakes_chars, "ROMEO: ", temperature=1.0))

ROMEO: Eveating soul;
And, you; pretagution; anitherous latter un,
Hast onew if he'd impnation a myselff-I been it is hall me.

HARTING:
I'll buties and roseingerngle pried's whomemone have art the Lounds.

ISABELLA:
Your whilst I shall tale! I maye the criem,
I break to brake ho beat so by theme innone,
The noble of Fad you say he tow his hands of ary.

AUFlIRS:
'This worden by that men's 'ang of his world,
And cannot in me ROMUMEN:
Who, gonsue; in?-Gullly lasigale geed
The ittreen, by tent hem
And, you to churger beswer.

JROUPERLE:
Inall, net-not nothing toon him: has a fer a tooce:
To no be be objementing grecious Composh of his poor lettes,
honour'd ane to your thouse to One,
This soulch ann to snacious of.

WARMINY:
Some it ba tid Youkt to-
Wo whose scepfurd.

ESTORIONA:
'Tis nefe them.'
Of the sutether some hore canst the everty mish dram, porshy,
Or the consommonafles cail;
You like maven blood that seld-ind gaves a conor
He rmoms a wingle wit: he sensit in
Giglieve of sopprick

# Assignment
Answer the following questions and hand in your solution in Canvas before 23:59 on Friday, September 27th. Remember to save your file before uploading it.

## Question 1
The `temperature` parameter of `generate_text()`, defined earlier in the notebook, controls how predictable the generated text will be. The lower the temperature, the more the function will tend to append the most likely character (according to the model's prediction). A higher temperature introduces some randomness, leading to more unpredictable text.

The text we generated above used a temperature of 1.0. Try generating more text using the Shakespeare model:

(a) once using a temperature of 0.2 and

(b) again using a temperature of 0.8

and describe the difference.

In [21]:
# Your solution here
print("Generated text with temperature of 0.8")
print(generate_text(shakes_model, shakes_chars, "ROMEO: ", temperature=0.8))
print("Generated text with temperature of 0.2")
print(generate_text(shakes_model, shakes_chars, "ROMEO: ", temperature=0.2))

Generated text with temperature of 0.8
ROMEO: my leader, and it, both I faip for the matter?

PLORY OF GAGH:
Pencereate that so lobed in hairated store o' to the consule may
To send the fairing; and scine a crovented-age,
I am sidess were to benith thee, with that do I thee were.

First Censting upon the stright sporn,
Enter for the bony of the playsall heaven.
Hasting injuther, no sol of his with to your hath,
When cannot should to seaven coursey sound'd, him, they,
Take I day not that it, and handstake black it.

HARTICK:
To sause to be a monall.

ROMEO:
Well, gray to rest in marth in this hat's frater his gentle,
To and not the cittle and my cault
As was for therefore hast to at such anstand
Hast demestarse unstracions with not be rest you.

LUCIO:
I take come brother so me senet in the streatht;
Bid you sweet him it mean not from the follows rigot,
And that I hear indurk'd to the cknot his flive,
Allostes he is be hate his flatter bear me,
I that agand me; is the wall,
thes her by 

Since an high temperature produce more randomness than the lower ones, using the temperature of 0.8 we have shorter sentences than the other generates using as value 0.2. Seems to be less grammatical errors with lower temperature (e.g. 0.2) compared to higher one (e.g. 0.8).

## Question 2
NLTK's `names` corpus contains a list of approximately 8,000 English names. Train a new model on `names_raw` for at least 20 epochs using the `create_model(text, epochs=n)` function defined earlier. Use the trained model to generate a list of names (with the `generate_text` function defined earlier), starting with your own first name. Your name should not contain any non-English characters, and should end with an `\n`.

Print out the names that do not appear in the training data.

(a) Do you get any actual names (or at least names that sound plausible)?

In [11]:
# Don't modify this code cell
import nltk
from nltk.corpus import names
nltk.download('names')

# Print out a few examples
names_raw = names.raw()
names_unique = set(names_raw.split())
names_raw = "\n".join(names_unique)
print(names_raw.splitlines()[:5])

['Valina', 'Darsie', 'Verla', 'Moria', 'Adey']


[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.


In [28]:
# Your solution here
names_model, names_chars = create_model(names_raw, epochs=20)
print(generate_text(names_model, names_chars, "Giorgio\n", temperature=0.2))

55 unique characters
Epoch 1/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - loss: 3.9299
Epoch 2/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 3.7486
Epoch 3/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 3.1864
Epoch 4/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 2.8473
Epoch 5/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 2.5858
Epoch 6/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 2.4778
Epoch 7/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 2.4281
Epoch 8/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 2.3788
Epoch 9/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 2.3520
Epoch 10/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 2.3166
Epoch 11/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 2.2899
Epoch 12/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 2.2730
Epoch 13/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 2.2542
Epoch 14/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 2.2400
Epoch 15/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 2.2253
Epoch 16/20
8

Yes, I've got "Marie" like actual name using a temperature of 0.2, using higher ones the names that I've got has at the end a sound plausible

##Question 3
The size of the model can make a difference when it comes to performance. Create a new model that has twice the number of hidden units as the previous model and double the size of the embeddings.

(a) How does the performance change?

(b) What happens if you decrease these parameters?

In [35]:
# Your solution here
def create_powerful_model(text, epochs=3):
    dataset, vocab, examples_per_epoch, char_map = prepare_text(text)

    train_model = build_model(len(vocab), EMBEDDING_DIMENSION//2, RNN_UNITS//2, BATCH_SIZE)
    train_model.compile(optimizer='adam', loss=loss)

    train_model.fit(dataset, epochs=epochs)

    pred_model = build_model(len(vocab), EMBEDDING_DIMENSION//2, RNN_UNITS//2, batch_size=1)
    pred_model.build(input_shape=(1,100))
    pred_model.set_weights(train_model.get_weights())

    return pred_model, char_map


In [36]:
names_model, names_chars = create_powerful_model(names_raw, epochs=20)
print(generate_text(names_model, names_chars, "Giorgio\n", temperature=1))

55 unique characters
Epoch 1/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 3.9064
Epoch 2/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 3.3581
Epoch 3/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 3.1538
Epoch 4/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 3.0697
Epoch 5/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 2.8968
Epoch 6/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 2.6705
Epoch 7/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 2.5269
Epoch 8/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 2.4526
Epoch 9/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 2.4140
Epoch 10/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 2.3777
Epoch 11/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 2.3538
Epoch 12/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 2.3303
Epoch 13/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 2.3145
Epoch 14/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 2.2945
Epoch 15/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 2.2750
Epoch 16/20
8

With a model that has twice the number of hidden units and double the size of embedding the model has an improvement of performance even with 0.2 and with 1.0 as temperature.

Instead with a model with less hidden units and a lower size of embedding the model is faster to train It but has a great loss in perfomance

## Question 4
Transformer large language models can also generate text. The following code imports a pretrained GPT-2 model from Huggingface's Transformer library. This model can then be used directly to generate text, given a prompt as context. Alter the prompt to have the transformer model (GPT-2) generate an engaging story beginning using one of the following story starters:


*   It was the day the moon fell.
*   Am I in heaven?  What happened to me?
*   Wandering through the graveyard it felt like something was watching me.
*   Three of us.  We were the only ones left, the only ones to make it to the island.

There are several different methods to choose from to generate the text (as seen in the commented out lines below). Try out the different methods and play with the parameters. This [blogpost](https://huggingface.co/blog/how-to-generate) explains their differences.

(a) Which method has the best performance?

(b) Can GPT-2 generate Shakespere?

In [37]:
# Uncomment if transformers is not installed
!pip install transformers

In [38]:
# Do not modify this code
# https://huggingface.co/docs/transformers/main_classes/text_generation

from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("gpt2")

gpt2_model = AutoModelForCausalLM.from_pretrained("gpt2")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [39]:
# Do not modify this code

prompt = "Today I believe we can finally"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

outputs = gpt2_model.generate(input_ids, max_length=100) # Greedy search
#outputs = gpt2_model.generate(input_ids, max_length=100, num_beams=5, no_repeat_ngram_size=3, early_stopping=True) # Beam search
#outputs = gpt2_model.generate(input_ids, do_sample=True, max_length=100, top_k=0, temperature=0.7) # Sampling
#outputs = gpt2_model.generate(input_ids, do_sample=True, max_length=100, top_k=50) # Top-k
#outputs = gpt2_model.generate(input_ids, do_sample=True, max_length=100, top_k=50, top_p=0.92) # Top-p

tokenizer.batch_decode(outputs, skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Today I believe we can finally get to the point where we can make a difference in the lives of the people of the United States of America.\n\nI believe that we can make a difference in the lives of the people of the United States of America.\n\nI believe that we can make a difference in the lives of the people of the United States of America.\n\nI believe that we can make a difference in the lives of the people of the United States of America.\n\n']

In [47]:
# Your answer here
prompt = "Am I in heaven? What happened to me?"

input_ids = tokenizer(prompt, return_tensors='pt').input_ids
#outputs = gpt2_model.generate(input_ids, max_length=100) # Greedy search
#outputs = gpt2_model.generate(input_ids, max_length=100, num_beams=5, no_repeat_ngram_size=3, early_stopping=True) # Beam search
#outputs = gpt2_model.generate(input_ids, do_sample=True, max_length=100, top_k=0, temperature=0.7) # Sampling
outputs = gpt2_model.generate(input_ids, do_sample=True, max_length=100, top_k=50) # Top-k
#outputs = gpt2_model.generate(input_ids, do_sample=True, max_length=100, top_k=50, top_p=0.92) # Top-p

tokenizer.batch_decode(outputs, skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


["Am I in heaven? What happened to me?\n\nHe had a strong argument with me and his question should not have been asked. Since he was young there was no reason to question me or the answer to my question should not have been taken. I didn't dare ask it either.\n\nThe point of having a family for the long term would be to be a father. I knew it would be not good. It was a risk I would take even before I did what I felt"]

The model trained with a combination of top_k and top_p outperform the other ones creating sentences more structured and meaningful.

In [63]:
input_ids = tokenizer(shakespeare[125:225], return_tensors='pt').input_ids

outputs = gpt2_model.generate(input_ids, do_sample=True, max_length=100, top_k=50, top_p=0.92) # Top-p
# outputs = gpt2_model.generate(input_ids, do_sample=True, max_length=100, top_k=50) # Top-k


tokenizer.batch_decode(outputs, skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is churlish.\n\nYou know that a noble was not a child of Caius Marcius until he was forced to become a slave?\n\nWell, I say, the first that you hear that is from Caius.\n\nAll:\n\nResolved. resolved.\n\nFirst Citizen:\n\n']

Seems that GPT-2 is not able to generate shakespeare neither using the top-k and the combinaion of top-k and top-p